In [ ]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import pandas as pd

from data_loader_darts import get_dataloaders_simple
# from darts_search_bdp import train_darts_search_bdp
from darts_search_bdp import train_darts_search_bdp
from model_build import FinalNetwork
from cell_plot import plot_cell

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

In [3]:

# 1. Set random seed
set_seed(42)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 2. Load data
print("[INFO] Loading 50/50 split data...")
train_loader, val_loader, num_classes = get_dataloaders_simple(batch_size=32)
print(f"[INFO] DARTS will run on {len(train_loader.dataset.y)} train samples and {len(val_loader.dataset.y)} val samples")

[INFO] Loading 50/50 split data...
[DEBUG] Loaded ./PSG/SC4001E0.npz → 841 samples
[DEBUG] Loaded ./PSG/SC4002E0.npz → 1127 samples
[DEBUG] Loaded ./PSG/SC4011E0.npz → 1103 samples
[DEBUG] Loaded ./PSG/SC4012E0.npz → 1186 samples
[DEBUG] Loaded ./PSG/SC4021E0.npz → 1025 samples
[DEBUG] Loaded ./PSG/SC4022E0.npz → 1009 samples
[DEBUG] Loaded ./PSG/SC4031E0.npz → 952 samples
[DEBUG] Loaded ./PSG/SC4032E0.npz → 911 samples
[DEBUG] Loaded ./PSG/SC4041E0.npz → 1235 samples
[DEBUG] Loaded ./PSG/SC4042E0.npz → 1200 samples
[DEBUG] Loaded ./PSG/SC4051E0.npz → 672 samples
[DEBUG] Loaded ./PSG/SC4052E0.npz → 1246 samples
[DEBUG] Loaded ./PSG/SC4061E0.npz → 843 samples
[DEBUG] Loaded ./PSG/SC4062E0.npz → 1016 samples
[DEBUG] Loaded ./PSG/SC4071E0.npz → 976 samples
[DEBUG] Loaded ./PSG/SC4072E0.npz → 1273 samples
[DEBUG] Loaded ./PSG/SC4081E0.npz → 1134 samples
[DEBUG] Loaded ./PSG/SC4082E0.npz → 1054 samples
[DEBUG] Loaded ./PSG/SC4091E0.npz → 1132 samples
[DEBUG] Loaded ./PSG/SC4092E0.npz → 1105

In [4]:
# 3. Run DARTS search with pruning
print("[INFO] Running DARTS search with BDP...")
searched_genotype, pruned_train_loader, pruned_val_loader = train_darts_search_bdp(
    train_loader, val_loader, num_classes,
    epochs=25, prune_every=5, pt=0.05, pv=0.05,
    device=device
)

[INFO] Running DARTS search with BDP...

[Epoch 1/25] Starting...
[Epoch 1] Train Loss: 1.0660 | Acc: 0.5704 || Val Loss: 1.0277 | Acc: 0.5929

[Epoch 2/25] Starting...
[Epoch 2] Train Loss: 0.8630 | Acc: 0.6669 || Val Loss: 0.7655 | Acc: 0.6751

[Epoch 3/25] Starting...
[Epoch 3] Train Loss: 0.7609 | Acc: 0.7169 || Val Loss: 0.7263 | Acc: 0.7390

[Epoch 4/25] Starting...
[Epoch 4] Train Loss: 0.6786 | Acc: 0.7443 || Val Loss: 0.6234 | Acc: 0.7743

[Epoch 5/25] Starting...
[Epoch 5] Train Loss: 0.6301 | Acc: 0.7639 || Val Loss: 0.8818 | Acc: 0.6819
[Annealable Pruning] T = 1.2218
[Prune Epoch 5] Pruned 1057 train, 1057 val
[After Prune] Remaining train samples: 20097, val samples: 20097

[Epoch 6/25] Starting...
[Epoch 6] Train Loss: 0.6176 | Acc: 0.7682 || Val Loss: 0.5769 | Acc: 0.7913

[Epoch 7/25] Starting...
[Epoch 7] Train Loss: 0.5778 | Acc: 0.7850 || Val Loss: 0.6124 | Acc: 0.7707

[Epoch 8/25] Starting...
[Epoch 8] Train Loss: 0.5711 | Acc: 0.7888 || Val Loss: 0.5566 | Acc: 0.

In [2]:
from collections import namedtuple

Genotype = namedtuple('Genotype', 'normal normal_concat reduce reduce_concat')

searched_genotype = Genotype(
    normal=[
        ('sep_conv_1x5', 0),
        ('sep_conv_1x5', 1),
        ('sep_conv_1x5', 0),
        ('max_pool_3x3', 2),
        ('max_pool_3x3', 2),
        ('max_pool_3x3', 3),
        ('sep_conv_1x5', 0),
        ('sep_conv_1x5', 1),
        ('sep_conv_1x5', 1),
        ('max_pool_3x3', 5),
        ('dil_conv_1x5', 0),
        ('dil_conv_1x5', 0)
    ],
    normal_concat=[0, 1, 2, 3, 4],

    reduce=[
        ('sep_conv_1x3', 0),
        ('sep_conv_1x5', 1),
        ('dil_conv_1x5', 0),
        ('dil_conv_1x5', 2),
        ('max_pool_3x3', 0),
        ('max_pool_3x3', 3),
        ('max_pool_3x3', 0),
        ('max_pool_3x3', 4),
        ('max_pool_3x3', 5),
        ('avg_pool_3x3', 0)
    ],
    reduce_concat=[0, 1, 2, 3, 4]
)


In [ ]:
import torch
import pandas as pd
import numpy as np

print("[INFO] Running 5-Fold Cross Validation on pruned data...")

# === Hàm trích xuất toàn bộ dữ liệu từ DataLoader ===
def extract_from_loader(loader):
    X_list, y_list = [], []
    for x, y in loader:
        X_list.append(x.cpu())
        y_list.append(y.cpu())
    return torch.cat(X_list, dim=0), torch.cat(y_list, dim=0)

# === Trích xuất X, y từ train và val loader ===
X_train_all, y_train_all = extract_from_loader(pruned_train_loader)
X_val_all,   y_val_all   = extract_from_loader(pruned_val_loader)

# Gộp train + val
X_all = torch.cat([X_train_all, X_val_all], dim=0)  # shape: (N, C, T) hoặc (N, T)
y_all = torch.cat([y_train_all, y_val_all], dim=0)  # shape: (N,)

# === Chuyển về numpy ===
X_np = X_all.numpy()
y_np = y_all.numpy().reshape(-1, 1)

# === Reshape X về (N, D) nếu cần (flatten nếu có chiều phụ) ===
if X_np.ndim > 2:
    X_np = X_np.reshape(X_np.shape[0], -1)  # (N, D)

# ❌ KHÔNG chuẩn hóa, giữ nguyên raw X_np

# === Ghép lại X và y ===
data_np = np.hstack((X_np, y_np))  # shape: (N, D+1)

# === Tạo DataFrame và lưu CSV ===
num_features = X_np.shape[1]
column_names = [f"feature_{i}" for i in range(num_features)] + ["label"]
df = pd.DataFrame(data_np, columns=column_names)

df.to_csv("pruned_dataset.csv", index=False)
print("✅ Đã lưu dữ liệu gốc (KHÔNG chuẩn hóa) vào 'pruned_dataset.csv'")


[INFO] Running 5-Fold Cross Validation on pruned data...
✅ Đã lưu dữ liệu gốc (KHÔNG chuẩn hóa) vào 'pruned_dataset.csv'


In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values
unique, counts = np.unique(y_np, return_counts=True)

print("\n[INFO] Number of samples per class:")
for label, count in zip(unique, counts):
    print(f"  Class {label}: {count} samples")



[INFO] Number of samples per class:
  Class 0: 6570 samples
  Class 1: 2504 samples
  Class 2: 14965 samples
  Class 3: 4274 samples
  Class 4: 6638 samples


In [ ]:
!sudo apt update
!sudo apt install -y graphviz graphviz-dev
!pip install pygraphviz
# 4. Visualize searched cells
print("[INFO] Visualizing searched cells...")
plot_cell(searched_genotype, 'normal')
plot_cell(searched_genotype, 'reduce')

Hit:1 https://deb.nodesource.com/node_20.x nodistro InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
171 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libgraphviz-dev' instead of 'graphviz-dev'
graphviz is already the newest version (2.42.2-6ubuntu0.1).
libgraphviz-dev is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 171 not upgraded.
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Installing build dep

/home/work/Duc/NASEEG1_Final/NASEEG/SleepC/sleep_nas/cell_plot.py:26: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


In [ ]:
# Post-search filter pruning based on information capacity and independence (Stage 2)
# === Entropy-based pruning ===
def compute_filter_entropy(weight_tensor):
    entropy_list = []
    for filt in weight_tensor:
        filt_flat = filt.view(filt.size(0), -1)
        norms = torch.norm(filt_flat, dim=1) + 1e-6
        p = norms / norms.sum()
        entropy = -torch.sum(p * torch.log2(p))
        entropy_list.append(entropy.item())
    return entropy_list

def prune_model_entropy(model, prune_ratio=0.5):
    for name, module in model.named_modules():
        if isinstance(module, nn.Conv1d):
            weight = module.weight.data.detach().cpu()
            entropy = compute_filter_entropy(weight)
            entropy_tensor = torch.tensor(entropy)
            k = int((1 - prune_ratio) * len(entropy))
            topk_indices = torch.topk(entropy_tensor, k=k).indices
            mask = torch.zeros_like(entropy_tensor)
            mask[topk_indices] = 1.0
            full_mask = mask[:, None, None].expand_as(weight).to(module.weight.device)
            module.weight.data *= full_mask
    return model

def count_pruned_weights(model):
    total, nonzero = 0, 0
    for module in model.modules():
        if isinstance(module, (nn.Conv1d, nn.Linear)):
            w = module.weight.data
            total += w.numel()
            nonzero += w.nonzero().size(0)
    zero = total - nonzero
    print(f"[INFO] Total weights: {total}")
    print(f"[INFO] Non-zero weights: {nonzero}")
    print(f"[INFO] Pruned weights: {zero}")
    print(f"[INFO] Pruned ratio: {100 * zero / total:.2f}%")

def evaluate_metrics(y_true, y_pred, num_classes):
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_true, y_pred, average=None, zero_division=0)
    rec = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1s = f1_score(y_true, y_pred, average=None, zero_division=0)
    gmeans = []

    for c in range(num_classes):
        tp = np.sum((y_pred == c) & (y_true == c))
        fn = np.sum((y_pred != c) & (y_true == c))
        recall_c = tp / (tp + fn) if (tp + fn) > 0 else 0
        gmeans.append(recall_c)

    mgm = np.sqrt(np.prod(gmeans)) if np.all(np.array(gmeans) > 0) else 0.0
    cm = confusion_matrix(y_true, y_pred)
    return acc, mf1, mgm, prec, rec, f1s, gmeans, cm

In [ ]:
#9_1
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from model_build import FinalNetwork

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

# Z-score normalization
X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Evaluation Metric Function ====
def evaluate_metrics(y_true, y_pred, num_classes):
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_true, y_pred, average=None, zero_division=0)
    rec = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1s = f1_score(y_true, y_pred, average=None, zero_division=0)

    gmeans = []
    for c in range(num_classes):
        tp = np.sum((y_pred == c) & (y_true == c))
        fn = np.sum((y_pred != c) & (y_true == c))
        tn = np.sum((y_pred != c) & (y_true != c))
        fp = np.sum((y_pred == c) & (y_true != c))
        gm = np.sqrt((tp / (tp + fn + 1e-6)) * (tn / (tn + fp + 1e-6)))
        gmeans.append(gm)
    cm = confusion_matrix(y_true, y_pred)
    return acc, mf1, np.mean(gmeans), prec, rec, f1s, gmeans, cm

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=47)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    model = FinalNetwork(C=8, num_classes=num_classes, layers=9, genotype=searched_genotype).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOP counter

    for epoch in range(50):
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] "
              f"Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(val_true), np.array(val_pred), num_classes)
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 15:
                print(f"[Early Stopping] No improvement in 15 epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[Fold 1 | Epoch 1] Train Loss: 0.5451 | Train Acc: 0.8001 | Val Loss: 0.6944 | Val Acc: 0.7412
[Fold 1 | Epoch 2] Train Loss: 0.4539 | Train Acc: 0.8337 | Val Loss: 0.6234 | Val Acc: 0.7750
[Fold 1 | Epoch 3] Train Loss: 0.4392 | Train Acc: 0.8378 | Val Loss: 0.4072 | Val Acc: 0.8515
[Fold 1 | Epoch 4] Train Loss: 0.4238 | Train Acc: 0.8445 | Val Loss: 0.4571 | Val Acc: 0.8346
[Fold 1 | Epoch 5] Train Loss: 0.4201 | Train Acc: 0.8448 | Val Loss: 0.4132 | Val Acc: 0.8552
[Fold 1 | Epoch 6] Train Loss: 0.4096 | Train Acc: 0.8501 | Val Loss: 0.4279 | Val Acc: 0.8441
[Fold 1 | Epoch 7] Train Loss: 0.4097 | Train Acc: 0.8486 | Val Loss: 0.4284 | Val Acc: 0.8429
[Fold 1 | Epoch 8] Train Loss: 0.3994 | Train Acc: 0.8524 | Val Loss: 0.4099 | Val Acc: 0.8489
[Fold 1 | Epoch 9] Train Loss: 0.3920 | Train Acc: 0.8572 | Val Loss: 0.4235 | Val Acc: 0.8435
[Fold 1 | Epoch 10] Train Loss: 0.3849 | Train Acc: 0.8584 | Val Loss: 0.3952 | Val Acc: 0.8528
[Fold 1 | Epoch 11] Train Los

[Fold 3 | Epoch 3] Train Loss: 0.4437 | Train Acc: 0.8365 | Val Loss: 0.5133 | Val Acc: 0.8197
[Fold 3 | Epoch 4] Train Loss: 0.4272 | Train Acc: 0.8433 | Val Loss: 0.5355 | Val Acc: 0.7956
[Fold 3 | Epoch 5] Train Loss: 0.4211 | Train Acc: 0.8447 | Val Loss: 0.4847 | Val Acc: 0.8136
[Fold 3 | Epoch 6] Train Loss: 0.4102 | Train Acc: 0.8495 | Val Loss: 0.4942 | Val Acc: 0.8232
[Fold 3 | Epoch 7] Train Loss: 0.4065 | Train Acc: 0.8487 | Val Loss: 0.3988 | Val Acc: 0.8564
[Fold 3 | Epoch 8] Train Loss: 0.3979 | Train Acc: 0.8542 | Val Loss: 0.4107 | Val Acc: 0.8459
[Fold 3 | Epoch 9] Train Loss: 0.3924 | Train Acc: 0.8573 | Val Loss: 0.4076 | Val Acc: 0.8482
[Fold 3 | Epoch 10] Train Loss: 0.3855 | Train Acc: 0.8572 | Val Loss: 0.3837 | Val Acc: 0.8535
[Fold 3 | Epoch 11] Train Loss: 0.3828 | Train Acc: 0.8569 | Val Loss: 0.4953 | Val Acc: 0.8295
[Fold 3 | Epoch 12] Train Loss: 0.3778 | Train Acc: 0.8584 | Val Loss: 0.3814 | Val Acc: 0.8558
[Fold 3 | Epoch 13] Train Loss: 0.3699 | Train 

[Fold 5 | Epoch 5] Train Loss: 0.4201 | Train Acc: 0.8459 | Val Loss: 0.4205 | Val Acc: 0.8539
[Fold 5 | Epoch 6] Train Loss: 0.4103 | Train Acc: 0.8505 | Val Loss: 0.4824 | Val Acc: 0.8222
[Fold 5 | Epoch 7] Train Loss: 0.4045 | Train Acc: 0.8500 | Val Loss: 0.4481 | Val Acc: 0.8372
[Fold 5 | Epoch 8] Train Loss: 0.4010 | Train Acc: 0.8519 | Val Loss: 0.4000 | Val Acc: 0.8548
[Fold 5 | Epoch 9] Train Loss: 0.3925 | Train Acc: 0.8540 | Val Loss: 0.3935 | Val Acc: 0.8531
[Fold 5 | Epoch 10] Train Loss: 0.3847 | Train Acc: 0.8574 | Val Loss: 0.3780 | Val Acc: 0.8615
[Fold 5 | Epoch 11] Train Loss: 0.3790 | Train Acc: 0.8595 | Val Loss: 0.4379 | Val Acc: 0.8445
[Fold 5 | Epoch 12] Train Loss: 0.3747 | Train Acc: 0.8623 | Val Loss: 0.3980 | Val Acc: 0.8502
[Fold 5 | Epoch 13] Train Loss: 0.3697 | Train Acc: 0.8624 | Val Loss: 0.4387 | Val Acc: 0.8454
[Fold 5 | Epoch 14] Train Loss: 0.3643 | Train Acc: 0.8662 | Val Loss: 0.4430 | Val Acc: 0.8359
[Fold 5 | Epoch 15] Train Loss: 0.3574 | Trai

In [ ]:
#11_1
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from model_build import FinalNetwork

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

# Z-score normalization
X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Evaluation Metric Function ====
def evaluate_metrics(y_true, y_pred, num_classes):
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_true, y_pred, average=None, zero_division=0)
    rec = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1s = f1_score(y_true, y_pred, average=None, zero_division=0)

    gmeans = []
    for c in range(num_classes):
        tp = np.sum((y_pred == c) & (y_true == c))
        fn = np.sum((y_pred != c) & (y_true == c))
        tn = np.sum((y_pred != c) & (y_true != c))
        fp = np.sum((y_pred == c) & (y_true != c))
        gm = np.sqrt((tp / (tp + fn + 1e-6)) * (tn / (tn + fp + 1e-6)))
        gmeans.append(gm)
    cm = confusion_matrix(y_true, y_pred)
    return acc, mf1, np.mean(gmeans), prec, rec, f1s, gmeans, cm

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=47)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    model = FinalNetwork(C=8, num_classes=num_classes, layers=11, genotype=searched_genotype).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOP counter

    for epoch in range(50):
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] "
              f"Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(val_true), np.array(val_pred), num_classes)
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 15:
                print(f"[Early Stopping] No improvement in 15 epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[Fold 1 | Epoch 1] Train Loss: 0.5439 | Train Acc: 0.7987 | Val Loss: 0.4998 | Val Acc: 0.8218
[Fold 1 | Epoch 2] Train Loss: 0.4643 | Train Acc: 0.8302 | Val Loss: 0.5252 | Val Acc: 0.8126
[Fold 1 | Epoch 3] Train Loss: 0.4393 | Train Acc: 0.8381 | Val Loss: 0.7205 | Val Acc: 0.7468
[Fold 1 | Epoch 4] Train Loss: 0.4313 | Train Acc: 0.8399 | Val Loss: 0.4062 | Val Acc: 0.8448
[Fold 1 | Epoch 5] Train Loss: 0.4185 | Train Acc: 0.8452 | Val Loss: 0.4156 | Val Acc: 0.8409
[Fold 1 | Epoch 6] Train Loss: 0.4128 | Train Acc: 0.8497 | Val Loss: 0.4137 | Val Acc: 0.8431
[Fold 1 | Epoch 7] Train Loss: 0.4080 | Train Acc: 0.8494 | Val Loss: 0.4199 | Val Acc: 0.8445
[Fold 1 | Epoch 8] Train Loss: 0.3996 | Train Acc: 0.8518 | Val Loss: 0.4026 | Val Acc: 0.8447
[Fold 1 | Epoch 9] Train Loss: 0.3957 | Train Acc: 0.8540 | Val Loss: 0.4579 | Val Acc: 0.8321
[Fold 1 | Epoch 10] Train Loss: 0.3884 | Train Acc: 0.8563 | Val Loss: 0.3907 | Val Acc: 0.8532
[Fold 1 | Epoch 11] Train Los

[Fold 3 | Epoch 14] Train Loss: 0.3681 | Train Acc: 0.8625 | Val Loss: 0.3858 | Val Acc: 0.8536
[Fold 3 | Epoch 15] Train Loss: 0.3650 | Train Acc: 0.8639 | Val Loss: 0.3943 | Val Acc: 0.8529
[Fold 3 | Epoch 16] Train Loss: 0.3631 | Train Acc: 0.8645 | Val Loss: 0.4019 | Val Acc: 0.8546
[Fold 3 | Epoch 17] Train Loss: 0.3559 | Train Acc: 0.8669 | Val Loss: 0.4438 | Val Acc: 0.8358
[Fold 3 | Epoch 18] Train Loss: 0.3539 | Train Acc: 0.8674 | Val Loss: 0.4523 | Val Acc: 0.8352
[Fold 3 | Epoch 19] Train Loss: 0.3490 | Train Acc: 0.8679 | Val Loss: 0.4036 | Val Acc: 0.8605
[Fold 3 | Epoch 20] Train Loss: 0.3403 | Train Acc: 0.8695 | Val Loss: 0.4247 | Val Acc: 0.8482
[Fold 3 | Epoch 21] Train Loss: 0.3356 | Train Acc: 0.8749 | Val Loss: 0.4024 | Val Acc: 0.8552
[Fold 3 | Epoch 22] Train Loss: 0.3304 | Train Acc: 0.8746 | Val Loss: 0.4003 | Val Acc: 0.8478
[Fold 3 | Epoch 23] Train Loss: 0.3243 | Train Acc: 0.8792 | Val Loss: 0.5004 | Val Acc: 0.8315
[Fold 3 | Epoch 24] Train Loss: 0.3202 |

[Fold 5 | Epoch 6] Train Loss: 0.4095 | Train Acc: 0.8478 | Val Loss: 0.4390 | Val Acc: 0.8280
[Fold 5 | Epoch 7] Train Loss: 0.4044 | Train Acc: 0.8518 | Val Loss: 0.4120 | Val Acc: 0.8488
[Fold 5 | Epoch 8] Train Loss: 0.4041 | Train Acc: 0.8492 | Val Loss: 0.3990 | Val Acc: 0.8587
[Fold 5 | Epoch 9] Train Loss: 0.3971 | Train Acc: 0.8527 | Val Loss: 0.3973 | Val Acc: 0.8512
[Fold 5 | Epoch 10] Train Loss: 0.3878 | Train Acc: 0.8556 | Val Loss: 0.3838 | Val Acc: 0.8599
[Fold 5 | Epoch 11] Train Loss: 0.3854 | Train Acc: 0.8570 | Val Loss: 0.4331 | Val Acc: 0.8368
[Fold 5 | Epoch 12] Train Loss: 0.3786 | Train Acc: 0.8598 | Val Loss: 0.3990 | Val Acc: 0.8548
[Fold 5 | Epoch 13] Train Loss: 0.3709 | Train Acc: 0.8611 | Val Loss: 0.3860 | Val Acc: 0.8534
[Fold 5 | Epoch 14] Train Loss: 0.3754 | Train Acc: 0.8605 | Val Loss: 0.3991 | Val Acc: 0.8546
[Fold 5 | Epoch 15] Train Loss: 0.3644 | Train Acc: 0.8648 | Val Loss: 0.4038 | Val Acc: 0.8516
[Fold 5 | Epoch 16] Train Loss: 0.3620 | Tra

In [ ]:
#p11_2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    # === Build and Prune Model ===
    model_unpruned = FinalNetwork(C=8, num_classes=num_classes, layers=11, genotype=searched_genotype).to(device)
    total_params_before = sum(p.numel() for p in model_unpruned.parameters())
    print(f"[INFO] Total parameters BEFORE pruning: {total_params_before:,}")

    model = prune_model_entropy(model_unpruned, prune_ratio=0.5)
    nonzero_params_after = sum((p != 0).sum().item() for p in model.parameters())
    zero_params = total_params_before - nonzero_params_after
    pruned_ratio = 100 * zero_params / total_params_before
    print(f"[INFO] Non-zero parameters AFTER pruning: {nonzero_params_after:,}")
    print(f"[INFO] Pruned parameters: {zero_params:,} ({pruned_ratio:.2f}%)")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    criterion = nn.CrossEntropyLoss()

    # Dataloader
    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOPPING COUNTER

    for epoch in range(50):  # Max epochs
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(
                np.array(val_true), np.array(val_pred), num_classes
            )
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 15:
                print(f"[Early Stopping] No improvement in 15 consecutive epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    # Gộp toàn bộ val_true và val_pred để đánh giá toàn bộ tập sau K-Fold
    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[INFO] Total parameters BEFORE pruning: 132,989
[INFO] Non-zero parameters AFTER pruning: 66,977
[INFO] Pruned parameters: 66,012 (49.64%)
[Fold 1 | Epoch 1] Train Loss: 0.6034 | Train Acc: 0.7771 | Val Loss: 0.5232 | Val Acc: 0.8186
[Fold 1 | Epoch 2] Train Loss: 0.4830 | Train Acc: 0.8211 | Val Loss: 0.4411 | Val Acc: 0.8434
[Fold 1 | Epoch 3] Train Loss: 0.4625 | Train Acc: 0.8298 | Val Loss: 0.5458 | Val Acc: 0.7970
[Fold 1 | Epoch 4] Train Loss: 0.4490 | Train Acc: 0.8327 | Val Loss: 0.5270 | Val Acc: 0.8130
[Fold 1 | Epoch 5] Train Loss: 0.4343 | Train Acc: 0.8394 | Val Loss: 0.4235 | Val Acc: 0.8425
[Fold 1 | Epoch 6] Train Loss: 0.4269 | Train Acc: 0.8433 | Val Loss: 0.5290 | Val Acc: 0.8344
[Fold 1 | Epoch 7] Train Loss: 0.4248 | Train Acc: 0.8425 | Val Loss: 0.4528 | Val Acc: 0.8284
[Fold 1 | Epoch 8] Train Loss: 0.4094 | Train Acc: 0.8480 | Val Loss: 0.3933 | Val Acc: 0.8591
[Fold 1 | Epoch 9] Train Loss: 0.4091 | Train Acc: 0.8494 | Val Loss: 0.5185 | Va

[INFO] Non-zero parameters AFTER pruning: 66,977
[INFO] Pruned parameters: 66,012 (49.64%)
[Fold 3 | Epoch 1] Train Loss: 0.5860 | Train Acc: 0.7832 | Val Loss: 0.6524 | Val Acc: 0.7549
[Fold 3 | Epoch 2] Train Loss: 0.4786 | Train Acc: 0.8223 | Val Loss: 0.4644 | Val Acc: 0.8217
[Fold 3 | Epoch 3] Train Loss: 0.4556 | Train Acc: 0.8335 | Val Loss: 0.4725 | Val Acc: 0.8283
[Fold 3 | Epoch 4] Train Loss: 0.4422 | Train Acc: 0.8368 | Val Loss: 0.5546 | Val Acc: 0.7970
[Fold 3 | Epoch 5] Train Loss: 0.4360 | Train Acc: 0.8388 | Val Loss: 0.4530 | Val Acc: 0.8319
[Fold 3 | Epoch 6] Train Loss: 0.4253 | Train Acc: 0.8428 | Val Loss: 0.4581 | Val Acc: 0.8272
[Fold 3 | Epoch 7] Train Loss: 0.4172 | Train Acc: 0.8459 | Val Loss: 0.4062 | Val Acc: 0.8489
[Fold 3 | Epoch 8] Train Loss: 0.4124 | Train Acc: 0.8454 | Val Loss: 0.4092 | Val Acc: 0.8485
[Fold 3 | Epoch 9] Train Loss: 0.4075 | Train Acc: 0.8477 | Val Loss: 0.3924 | Val Acc: 0.8548
[Fold 3 | Epoch 10] Train Loss: 0.4061 | Train Acc: 0.

[INFO] Non-zero parameters AFTER pruning: 66,977
[INFO] Pruned parameters: 66,012 (49.64%)
[Fold 5 | Epoch 1] Train Loss: 0.6023 | Train Acc: 0.7786 | Val Loss: 0.6064 | Val Acc: 0.7884
[Fold 5 | Epoch 2] Train Loss: 0.4865 | Train Acc: 0.8216 | Val Loss: 0.4873 | Val Acc: 0.8235
[Fold 5 | Epoch 3] Train Loss: 0.4615 | Train Acc: 0.8316 | Val Loss: 0.4647 | Val Acc: 0.8270
[Fold 5 | Epoch 4] Train Loss: 0.4465 | Train Acc: 0.8375 | Val Loss: 0.4419 | Val Acc: 0.8343
[Fold 5 | Epoch 5] Train Loss: 0.4417 | Train Acc: 0.8369 | Val Loss: 0.4422 | Val Acc: 0.8332
[Fold 5 | Epoch 6] Train Loss: 0.4270 | Train Acc: 0.8444 | Val Loss: 0.4549 | Val Acc: 0.8316
[Fold 5 | Epoch 7] Train Loss: 0.4212 | Train Acc: 0.8429 | Val Loss: 0.4037 | Val Acc: 0.8492
[Fold 5 | Epoch 8] Train Loss: 0.4188 | Train Acc: 0.8474 | Val Loss: 0.4215 | Val Acc: 0.8402
[Fold 5 | Epoch 9] Train Loss: 0.4174 | Train Acc: 0.8485 | Val Loss: 0.4080 | Val Acc: 0.8485
[Fold 5 | Epoch 10] Train Loss: 0.4072 | Train Acc: 0.

In [ ]:
#p7_1
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt
from model_build import FinalNetwork

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

# Z-score normalization
X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== Evaluation Metric Function ====
def evaluate_metrics(y_true, y_pred, num_classes):
    acc = accuracy_score(y_true, y_pred)
    mf1 = f1_score(y_true, y_pred, average='macro', zero_division=0)
    prec = precision_score(y_true, y_pred, average=None, zero_division=0)
    rec = recall_score(y_true, y_pred, average=None, zero_division=0)
    f1s = f1_score(y_true, y_pred, average=None, zero_division=0)

    gmeans = []
    for c in range(num_classes):
        tp = np.sum((y_pred == c) & (y_true == c))
        fn = np.sum((y_pred != c) & (y_true == c))
        tn = np.sum((y_pred != c) & (y_true != c))
        fp = np.sum((y_pred == c) & (y_true != c))
        gm = np.sqrt((tp / (tp + fn + 1e-6)) * (tn / (tn + fp + 1e-6)))
        gmeans.append(gm)
    cm = confusion_matrix(y_true, y_pred)
    return acc, mf1, np.mean(gmeans), prec, rec, f1s, gmeans, cm

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=47)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    model = FinalNetwork(C=8, num_classes=num_classes, layers=7, genotype=searched_genotype).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0055)
    criterion = nn.CrossEntropyLoss()

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOP counter

    for epoch in range(50):
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] "
              f"Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(val_true), np.array(val_pred), num_classes)
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 15:
                print(f"[Early Stopping] No improvement in 10 epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[Fold 1 | Epoch 1] Train Loss: 0.5272 | Train Acc: 0.8049 | Val Loss: 0.4331 | Val Acc: 0.8409
[Fold 1 | Epoch 2] Train Loss: 0.4584 | Train Acc: 0.8326 | Val Loss: 0.4424 | Val Acc: 0.8361
[Fold 1 | Epoch 3] Train Loss: 0.4330 | Train Acc: 0.8410 | Val Loss: 0.4124 | Val Acc: 0.8518
[Fold 1 | Epoch 4] Train Loss: 0.4243 | Train Acc: 0.8429 | Val Loss: 0.4136 | Val Acc: 0.8487
[Fold 1 | Epoch 5] Train Loss: 0.4116 | Train Acc: 0.8492 | Val Loss: 0.4178 | Val Acc: 0.8462
[Fold 1 | Epoch 6] Train Loss: 0.4012 | Train Acc: 0.8527 | Val Loss: 0.3751 | Val Acc: 0.8572
[Fold 1 | Epoch 7] Train Loss: 0.3936 | Train Acc: 0.8540 | Val Loss: 0.4330 | Val Acc: 0.8482
[Fold 1 | Epoch 8] Train Loss: 0.3957 | Train Acc: 0.8555 | Val Loss: 0.3718 | Val Acc: 0.8560
[Fold 1 | Epoch 9] Train Loss: 0.3813 | Train Acc: 0.8574 | Val Loss: 0.3744 | Val Acc: 0.8610
[Fold 1 | Epoch 10] Train Loss: 0.3782 | Train Acc: 0.8604 | Val Loss: 0.4112 | Val Acc: 0.8484
[Fold 1 | Epoch 11] Train Los

[Fold 3 | Epoch 26] Train Loss: 0.2833 | Train Acc: 0.8923 | Val Loss: 0.4166 | Val Acc: 0.8538
[Fold 3 | Epoch 27] Train Loss: 0.2751 | Train Acc: 0.8977 | Val Loss: 0.4863 | Val Acc: 0.8435
[Early Stopping] No improvement in 10 epochs. Stopping at epoch 27.

===== BEST RESULT FOR FOLD 3 =====
Best Epoch: 12
ACC: 0.8597 | MF1: 0.8013 | G-Mean: 0.8649
[Class 0] Prec: 0.9138 | Rec: 0.9094 | F1: 0.9116 | GM: 0.9445
[Class 1] Prec: 0.5108 | Rec: 0.4243 | F1: 0.4635 | GM: 0.6411
[Class 2] Prec: 0.8868 | Rec: 0.9208 | F1: 0.9035 | GM: 0.9168
[Class 3] Prec: 0.9340 | Rec: 0.8783 | F1: 0.9053 | GM: 0.9333
[Class 4] Prec: 0.8150 | Rec: 0.8299 | F1: 0.8224 | GM: 0.8889

===== Fold 4 =====
[Fold 4 | Epoch 1] Train Loss: 0.5518 | Train Acc: 0.7980 | Val Loss: 0.4928 | Val Acc: 0.8180
[Fold 4 | Epoch 2] Train Loss: 0.4578 | Train Acc: 0.8332 | Val Loss: 0.4837 | Val Acc: 0.8269
[Fold 4 | Epoch 3] Train Loss: 0.4310 | Train Acc: 0.8403 | Val Loss: 0.4744 | Val Acc: 0.8285
[Fold 4 | Epoch 4] Train L

In [ ]:
#p7_2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=49)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    # === Build and Prune Model ===
    model_unpruned = FinalNetwork(C=8, num_classes=num_classes, layers=7, genotype=searched_genotype).to(device)
    total_params_before = sum(p.numel() for p in model_unpruned.parameters())
    print(f"[INFO] Total parameters BEFORE pruning: {total_params_before:,}")

    model = prune_model_entropy(model_unpruned, prune_ratio=0.5)
    nonzero_params_after = sum((p != 0).sum().item() for p in model.parameters())
    zero_params = total_params_before - nonzero_params_after
    pruned_ratio = 100 * zero_params / total_params_before
    print(f"[INFO] Non-zero parameters AFTER pruning: {nonzero_params_after:,}")
    print(f"[INFO] Pruned parameters: {zero_params:,} ({pruned_ratio:.2f}%)")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0055)
    criterion = nn.CrossEntropyLoss()

    # Dataloader
    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOPPING COUNTER

    for epoch in range(50):  # Max epochs
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(
                np.array(val_true), np.array(val_pred), num_classes
            )
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 15:
                print(f"[Early Stopping] No improvement in 20 consecutive epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    # Gộp toàn bộ val_true và val_pred để đánh giá toàn bộ tập sau K-Fold
    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[INFO] Total parameters BEFORE pruning: 83,789
[INFO] Non-zero parameters AFTER pruning: 42,377
[INFO] Pruned parameters: 41,412 (49.42%)
[Fold 1 | Epoch 1] Train Loss: 0.5837 | Train Acc: 0.7809 | Val Loss: 0.4900 | Val Acc: 0.8216
[Fold 1 | Epoch 2] Train Loss: 0.4625 | Train Acc: 0.8293 | Val Loss: 0.4490 | Val Acc: 0.8405
[Fold 1 | Epoch 3] Train Loss: 0.4470 | Train Acc: 0.8356 | Val Loss: 0.4251 | Val Acc: 0.8468
[Fold 1 | Epoch 4] Train Loss: 0.4272 | Train Acc: 0.8416 | Val Loss: 0.5772 | Val Acc: 0.7913
[Fold 1 | Epoch 5] Train Loss: 0.4253 | Train Acc: 0.8416 | Val Loss: 0.4207 | Val Acc: 0.8461
[Fold 1 | Epoch 6] Train Loss: 0.4107 | Train Acc: 0.8462 | Val Loss: 0.5249 | Val Acc: 0.8213
[Fold 1 | Epoch 7] Train Loss: 0.4077 | Train Acc: 0.8465 | Val Loss: 0.3978 | Val Acc: 0.8611
[Fold 1 | Epoch 8] Train Loss: 0.4021 | Train Acc: 0.8496 | Val Loss: 0.4697 | Val Acc: 0.8298
[Fold 1 | Epoch 9] Train Loss: 0.3985 | Train Acc: 0.8497 | Val Loss: 0.4634 | Val

[Fold 3 | Epoch 12] Train Loss: 0.3885 | Train Acc: 0.8555 | Val Loss: 0.3906 | Val Acc: 0.8548
[Fold 3 | Epoch 13] Train Loss: 0.3865 | Train Acc: 0.8569 | Val Loss: 0.3746 | Val Acc: 0.8625
[Fold 3 | Epoch 14] Train Loss: 0.3778 | Train Acc: 0.8595 | Val Loss: 0.4078 | Val Acc: 0.8505
[Fold 3 | Epoch 15] Train Loss: 0.3806 | Train Acc: 0.8584 | Val Loss: 0.3906 | Val Acc: 0.8588
[Fold 3 | Epoch 16] Train Loss: 0.3695 | Train Acc: 0.8632 | Val Loss: 0.4101 | Val Acc: 0.8474
[Fold 3 | Epoch 17] Train Loss: 0.3684 | Train Acc: 0.8640 | Val Loss: 0.3840 | Val Acc: 0.8577
[Fold 3 | Epoch 18] Train Loss: 0.3652 | Train Acc: 0.8643 | Val Loss: 0.4382 | Val Acc: 0.8449
[Fold 3 | Epoch 19] Train Loss: 0.3606 | Train Acc: 0.8652 | Val Loss: 0.3780 | Val Acc: 0.8611
[Fold 3 | Epoch 20] Train Loss: 0.3539 | Train Acc: 0.8672 | Val Loss: 0.4307 | Val Acc: 0.8388
[Fold 3 | Epoch 21] Train Loss: 0.3502 | Train Acc: 0.8692 | Val Loss: 0.5560 | Val Acc: 0.7954
[Fold 3 | Epoch 22] Train Loss: 0.3532 |

[INFO] Total parameters BEFORE pruning: 83,789
[INFO] Non-zero parameters AFTER pruning: 42,377
[INFO] Pruned parameters: 41,412 (49.42%)
[Fold 5 | Epoch 1] Train Loss: 0.5801 | Train Acc: 0.7853 | Val Loss: 0.4778 | Val Acc: 0.8272
[Fold 5 | Epoch 2] Train Loss: 0.4783 | Train Acc: 0.8229 | Val Loss: 0.4465 | Val Acc: 0.8363
[Fold 5 | Epoch 3] Train Loss: 0.4569 | Train Acc: 0.8318 | Val Loss: 0.4263 | Val Acc: 0.8545
[Fold 5 | Epoch 4] Train Loss: 0.4402 | Train Acc: 0.8374 | Val Loss: 0.5106 | Val Acc: 0.8167
[Fold 5 | Epoch 5] Train Loss: 0.4325 | Train Acc: 0.8392 | Val Loss: 0.3965 | Val Acc: 0.8506
[Fold 5 | Epoch 6] Train Loss: 0.4206 | Train Acc: 0.8451 | Val Loss: 0.4051 | Val Acc: 0.8526
[Fold 5 | Epoch 7] Train Loss: 0.4165 | Train Acc: 0.8457 | Val Loss: 0.3920 | Val Acc: 0.8511
[Fold 5 | Epoch 8] Train Loss: 0.4100 | Train Acc: 0.8474 | Val Loss: 0.4003 | Val Acc: 0.8486
[Fold 5 | Epoch 9] Train Loss: 0.4065 | Train Acc: 0.8486 | Val Loss: 0.4261 | Val Acc: 0.8381
[Fold 5

In [ ]:
#p9_2
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from torch.utils.data import TensorDataset, DataLoader

# ==== Load and Normalize Data ====
df = pd.read_csv("pruned_dataset.csv")
X_np = df.drop(columns=["label"]).values
y_np = df["label"].values

X_mean = X_np.mean(axis=0)
X_std = np.where(X_np.std(axis=0) == 0, 1, X_np.std(axis=0))
X_z = (X_np - X_mean) / X_std
X_z = np.clip(X_z, -3, 3) / 3.0

X_all = torch.tensor(X_z, dtype=torch.float32).unsqueeze(1)
y_all = torch.tensor(y_np, dtype=torch.long)

num_classes = len(np.unique(y_np))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ==== K-Fold Training ====
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
all_val_true, all_val_pred = [], []

for fold_idx, (train_idx, val_idx) in enumerate(kfold.split(X_all)):
    print(f"\n===== Fold {fold_idx + 1} =====")

    # === Build and Prune Model ===
    model_unpruned = FinalNetwork(C=8, num_classes=num_classes, layers=9, genotype=searched_genotype).to(device)
    total_params_before = sum(p.numel() for p in model_unpruned.parameters())
    print(f"[INFO] Total parameters BEFORE pruning: {total_params_before:,}")

    model = prune_model_entropy(model_unpruned, prune_ratio=0.5)
    nonzero_params_after = sum((p != 0).sum().item() for p in model.parameters())
    zero_params = total_params_before - nonzero_params_after
    pruned_ratio = 100 * zero_params / total_params_before
    print(f"[INFO] Non-zero parameters AFTER pruning: {nonzero_params_after:,}")
    print(f"[INFO] Pruned parameters: {zero_params:,} ({pruned_ratio:.2f}%)")

    optimizer = torch.optim.Adam(model.parameters(), lr=0.0055)
    criterion = nn.CrossEntropyLoss()

    # Dataloader
    X_train, y_train = X_all[train_idx], y_all[train_idx]
    X_val, y_val = X_all[val_idx], y_all[val_idx]
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=64, shuffle=False)

    best_val_acc = 0.0
    best_result = {}
    no_improve_counter = 0  # ← EARLY STOPPING COUNTER

    for epoch in range(50):  # Max epochs
        model.train()
        train_loss = 0
        train_true, train_pred = [], []

        for x, y in train_loader:
            x, y = x.to(device).squeeze(-1), y.to(device)
            optimizer.zero_grad()
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_true.extend(y.cpu().numpy())
            train_pred.extend(output.argmax(dim=1).cpu().numpy())

        model.eval()
        val_loss = 0
        val_true, val_pred = [], []
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device).squeeze(-1), y.to(device)
                output = model(x)
                loss = criterion(output, y)
                val_loss += loss.item()
                val_true.extend(y.cpu().numpy())
                val_pred.extend(output.argmax(dim=1).cpu().numpy())

        train_acc = accuracy_score(train_true, train_pred)
        val_acc = accuracy_score(val_true, val_pred)

        print(f"[Fold {fold_idx+1} | Epoch {epoch+1}] Train Loss: {train_loss/len(train_loader):.4f} | "
              f"Train Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            no_improve_counter = 0  # reset counter
            acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(
                np.array(val_true), np.array(val_pred), num_classes
            )
            best_result = {
                'epoch': epoch + 1,
                'acc': acc,
                'mf1': mf1,
                'gmean': mgm,
                'prec': prec,
                'rec': rec,
                'f1': f1s,
                'gmean_class': gmeans,
                'cm': cm,
                'val_true': val_true,
                'val_pred': val_pred
            }
        else:
            no_improve_counter += 1
            if no_improve_counter >= 15:
                print(f"[Early Stopping] No improvement in 15 consecutive epochs. Stopping at epoch {epoch+1}.")
                break

    # === Print Best Result for Fold ===
    print(f"\n===== BEST RESULT FOR FOLD {fold_idx+1} =====")
    print(f"Best Epoch: {best_result['epoch']}")
    print(f"ACC: {best_result['acc']:.4f} | MF1: {best_result['mf1']:.4f} | G-Mean: {best_result['gmean']:.4f}")
    for i in range(num_classes):
        print(f"[Class {i}] Prec: {best_result['prec'][i]:.4f} | Rec: {best_result['rec'][i]:.4f} "
              f"| F1: {best_result['f1'][i]:.4f} | GM: {best_result['gmean_class'][i]:.4f}")

    # Gộp toàn bộ val_true và val_pred để đánh giá toàn bộ tập sau K-Fold
    all_val_true.extend(best_result['val_true'])
    all_val_pred.extend(best_result['val_pred'])

# ==== FINAL EVALUATION ON MERGED VAL SET ====
acc, mf1, mgm, prec, rec, f1s, gmeans, cm = evaluate_metrics(np.array(all_val_true), np.array(all_val_pred), num_classes)

print("\n===== FINAL EVALUATION ON MERGED TEST SET (AFTER K-FOLD) =====")
print(f"ACC: {acc:.4f} | MF1: {mf1:.4f} | G-Mean: {mgm:.4f}")
for i in range(num_classes):
    print(f"[Class {i}] Prec: {prec[i]:.4f} | Rec: {rec[i]:.4f} | F1: {f1s[i]:.4f} | GM: {gmeans[i]:.4f}")
print("Confusion Matrix:")
print(cm)



===== Fold 1 =====
[INFO] Total parameters BEFORE pruning: 94,813
[INFO] Non-zero parameters AFTER pruning: 47,889
[INFO] Pruned parameters: 46,924 (49.49%)
[Fold 1 | Epoch 1] Train Loss: 0.5657 | Train Acc: 0.7915 | Val Loss: 0.4886 | Val Acc: 0.8181
[Fold 1 | Epoch 2] Train Loss: 0.4837 | Train Acc: 0.8217 | Val Loss: 0.5204 | Val Acc: 0.7990
[Fold 1 | Epoch 3] Train Loss: 0.4581 | Train Acc: 0.8326 | Val Loss: 0.4442 | Val Acc: 0.8499
[Fold 1 | Epoch 4] Train Loss: 0.4487 | Train Acc: 0.8348 | Val Loss: 0.5055 | Val Acc: 0.8069
[Fold 1 | Epoch 5] Train Loss: 0.4393 | Train Acc: 0.8388 | Val Loss: 0.4291 | Val Acc: 0.8554
[Fold 1 | Epoch 6] Train Loss: 0.4318 | Train Acc: 0.8419 | Val Loss: 0.4160 | Val Acc: 0.8550
[Fold 1 | Epoch 7] Train Loss: 0.4264 | Train Acc: 0.8433 | Val Loss: 0.4008 | Val Acc: 0.8505
[Fold 1 | Epoch 8] Train Loss: 0.4211 | Train Acc: 0.8455 | Val Loss: 0.3976 | Val Acc: 0.8532
[Fold 1 | Epoch 9] Train Loss: 0.4134 | Train Acc: 0.8461 | Val Loss: 0.4130 | Val

[Fold 3 | Epoch 4] Train Loss: 0.4388 | Train Acc: 0.8380 | Val Loss: 0.4121 | Val Acc: 0.8512
[Fold 3 | Epoch 5] Train Loss: 0.4243 | Train Acc: 0.8438 | Val Loss: 0.5080 | Val Acc: 0.8139
[Fold 3 | Epoch 6] Train Loss: 0.4175 | Train Acc: 0.8468 | Val Loss: 0.4012 | Val Acc: 0.8484
[Fold 3 | Epoch 7] Train Loss: 0.4105 | Train Acc: 0.8460 | Val Loss: 0.4039 | Val Acc: 0.8449
[Fold 3 | Epoch 8] Train Loss: 0.4059 | Train Acc: 0.8510 | Val Loss: 0.4460 | Val Acc: 0.8335
[Fold 3 | Epoch 9] Train Loss: 0.4049 | Train Acc: 0.8497 | Val Loss: 0.4458 | Val Acc: 0.8474
[Fold 3 | Epoch 10] Train Loss: 0.3966 | Train Acc: 0.8527 | Val Loss: 0.4818 | Val Acc: 0.8300
[Fold 3 | Epoch 11] Train Loss: 0.3919 | Train Acc: 0.8543 | Val Loss: 0.4124 | Val Acc: 0.8471
[Fold 3 | Epoch 12] Train Loss: 0.3887 | Train Acc: 0.8549 | Val Loss: 0.4202 | Val Acc: 0.8433
[Fold 3 | Epoch 13] Train Loss: 0.3855 | Train Acc: 0.8574 | Val Loss: 0.3982 | Val Acc: 0.8565
[Fold 3 | Epoch 14] Train Loss: 0.3833 | Train

[Fold 5 | Epoch 16] Train Loss: 0.3845 | Train Acc: 0.8594 | Val Loss: 0.4278 | Val Acc: 0.8426
[Fold 5 | Epoch 17] Train Loss: 0.3866 | Train Acc: 0.8573 | Val Loss: 0.4106 | Val Acc: 0.8426
[Fold 5 | Epoch 18] Train Loss: 0.3806 | Train Acc: 0.8593 | Val Loss: 0.3871 | Val Acc: 0.8524
[Fold 5 | Epoch 19] Train Loss: 0.3745 | Train Acc: 0.8587 | Val Loss: 0.3759 | Val Acc: 0.8597
[Fold 5 | Epoch 20] Train Loss: 0.3706 | Train Acc: 0.8638 | Val Loss: 0.4185 | Val Acc: 0.8454
[Fold 5 | Epoch 21] Train Loss: 0.3626 | Train Acc: 0.8636 | Val Loss: 0.4270 | Val Acc: 0.8512
[Fold 5 | Epoch 22] Train Loss: 0.3640 | Train Acc: 0.8658 | Val Loss: 0.4384 | Val Acc: 0.8386
[Fold 5 | Epoch 23] Train Loss: 0.3583 | Train Acc: 0.8689 | Val Loss: 0.4064 | Val Acc: 0.8471
[Fold 5 | Epoch 24] Train Loss: 0.3599 | Train Acc: 0.8678 | Val Loss: 0.4138 | Val Acc: 0.8482
[Fold 5 | Epoch 25] Train Loss: 0.3504 | Train Acc: 0.8700 | Val Loss: 0.3692 | Val Acc: 0.8621
[Fold 5 | Epoch 26] Train Loss: 0.3474 |